In [6]:
import pandas as pd
import os

def load_clean_csv(path):
    df = pd.read_csv(path)
    # Drop unnamed columns (often index junk)
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df.columns = df.columns.str.lower()
    return df

# Reusable function
def load_and_index(path):
    df = load_clean_csv(path)
    return df.set_index('time') if 'time' in df.columns else df


In [10]:
simulink_dir = "../simulink"

# Reload + merge
simulink_p = load_and_index(os.path.join(simulink_dir, 'simulink_p.csv'))
simulink_q = load_and_index(os.path.join(simulink_dir, 'simulink_q.csv'))
simulink_vabc = load_and_index(os.path.join(simulink_dir, 'simulink_vabc.csv'))
simulink_Iabc = load_and_index(os.path.join(simulink_dir, 'simulink_Iabc.csv'))

simulink_all = pd.concat(
    [simulink_p, simulink_q, simulink_vabc, simulink_Iabc],
    axis=1
)

# Reset if you want 'time' as a column instead of index
simulink_all = simulink_all.reset_index()


In [11]:

pwr_dyn_dir = '../PowerDynamics'

# Load each file
pwr_p = load_and_index(os.path.join(pwr_dyn_dir, 'pwrdyn_bus15_p.csv'))
pwr_q = load_and_index(os.path.join(pwr_dyn_dir, 'pwrdyn_bus15_q.csv'))
pwr_v = load_and_index(os.path.join(pwr_dyn_dir, 'pwrdyn_bus15_v.csv'))
pwr_all = pd.concat([pwr_p, pwr_q, pwr_v], axis=1).reset_index()


In [13]:

pf_dir = '../PowerFactory'

# Helper function
def load_pf_csv(filename, col_names):
    path = os.path.join(pf_dir, filename)
    df = pd.read_csv(path, skiprows=2, header=None)
    df.columns = col_names
    return df

# Load each
pf_p     = load_pf_csv('pf_p.csv',     ['time', 'p'])
pf_q     = load_pf_csv('pf_q.csv',     ['time', 'q'])
pf_vabc  = load_pf_csv('pf_Vabc.csv',  ['time', 'v_a', 'v_b', 'v_c'])
pf_Iabc  = load_pf_csv('pf_Iabc.csv',  ['time', 'i_a', 'i_b', 'i_c'])

# Merge on time
pf_all = pf_p.merge(pf_q, on='time')\
             .merge(pf_vabc, on='time')\
             .merge(pf_Iabc, on='time')


In [14]:
# Trim all dataframes to time between 10 and 15
pf_all = pf_all[(pf_all['time'] >= 10) & (pf_all['time'] <= 15)]
pwr_all = pwr_all[(pwr_all['time'] >= 10) & (pwr_all['time'] <= 15)]
simulink_all = simulink_all[(simulink_all['time'] >= 10) & (simulink_all['time'] <= 15)]

In [25]:
# active and reactive power to kw for all
# PF data: MW → kW
pf_all['p_kw'] = pf_all['p'] * 1000

# PowerDynamics: pu → kW (S_base = 100 MVA)
S_base = 100e6  # 100 MVA
pwr_all['p_kw'] = pwr_all['p'] * S_base / 1000  # VA → W → kW

# Simulink: W → kW
simulink_all['p_kw'] = simulink_all['p'] / 1000

pf_all['q_kw'] = pf_all['q'] * 1000

# PowerDynamics: pu → kW (S_base = 100 MVA)
S_base = 100e6  # 100 MVA
pwr_all['q_kw'] = pwr_all['q'] * S_base / 1000  # VA → W → kW

# Simulink: W → kW
simulink_all['q_kw'] = simulink_all['q'] / 1000

In [32]:
# convert PowerFactory Iabc to pu

import numpy as np

S_base = 100e6       # 100 MVA
V_base = 132e3        # 132 kV line-to-line (adjust if needed)
I_base = S_base / (np.sqrt(3) * V_base)  # in Amps

# Convert kA to A
pf_all['i_a_pu'] = (pf_all['i_a'] * 1000) / I_base
pf_all['i_b_pu'] = (pf_all['i_b'] * 1000) / I_base
pf_all['i_c_pu'] = (pf_all['i_c'] * 1000) / I_base


In [33]:
pf_all

,time,p,q,v_a,v_b,v_c,i_a,i_b,i_c,p_kw,q_kw,i_a_pu,i_b_pu,i_c_pu
0,10.000000,0.006798,0.004608,-0.861350,-0.109652,0.971002,-0.000044,0.000005,0.000039,6.797902,4.608114,-0.000100,0.000012,0.000088
1,10.000051,0.007992,0.001884,-0.848933,-0.130041,0.978974,-0.000044,0.000005,0.000039,7.991508,1.884073,-0.000100,0.000012,0.000088
2,10.000118,0.007939,0.002089,-0.832010,-0.157117,0.989127,-0.000044,0.000005,0.000039,7.938775,2.088625,-0.000100,0.000012,0.000088
3,10.000218,0.007853,0.002392,-0.805997,-0.197347,1.003344,-0.000044,0.000005,0.000039,7.852735,2.391707,-0.000100,0.000012,0.000088
4,10.000318,0.007756,0.002691,-0.778853,-0.237265,1.016118,-0.000044,0.000005,0.000039,7.755542,2.691149,-0.000100,0.000012,0.000088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55000,14.999618,0.007362,0.003643,-0.720542,-0.249023,0.969565,-0.000048,0.000010,0.000037,7.361510,3.642749,-0.000110,0.000024,0.000086
55001,14.999718,0.007213,0.003929,-0.692404,-0.287183,0.979588,-0.000048,0.000010,0.000037,7.213307,3.929416,-0.000110,0.000024,0.000086
55002,14.999818,0.007053,0.004210,-0.663138,-0.324900,0.988038,-0.000048,0.000010,0.000037,7.053417,4.209895,-0.000110,0.000024,0.000086
55003,14.999918,0.006882,0.004484,-0.632802,-0.362174,0.994976,-0.000048,0.000010,0.000037,6.882384,4.484316,-0.000110,0.000024,0.000086


In [27]:
pwr_all

,time,p,q,v,p_kw,q_kw
2001,10.01,0.00010,0.287887,0.971211,10.000001,28788.738453
2002,10.01,0.00010,0.287887,0.971211,10.000001,28788.738453
2003,10.02,0.00010,0.287887,0.971211,10.000001,28788.738453
2004,10.02,0.00010,0.287887,0.971211,10.000001,28788.738453
2005,10.03,0.00010,0.287887,0.971211,10.000001,28788.738453
...,...,...,...,...,...,...
3002,14.98,-0.29611,0.274957,0.953687,-29611.005901,27495.665300
3003,14.99,-0.29361,0.274779,0.953781,-29361.028847,27477.946828
3004,14.99,-0.29361,0.274779,0.953781,-29361.028847,27477.946828
3005,15.00,-0.29113,0.274607,0.953875,-29113.048340,27460.700866


In [28]:
simulink_all

,time,p,q,v_a,v_b,v_c,i_a,i_b,i_c,p_kw,q_kw
100000,10.0000,10000.0,2.937182e-10,-2.038010e-13,-0.866025,0.866025,-1.413053e-13,-0.524864,0.524864,10.0,2.937182e-13
100001,10.0001,10000.0,-2.370009e-09,3.769018e-02,-0.884255,0.846565,2.284253e-02,-0.535912,0.513070,10.0,-2.370009e-12
100002,10.0002,10000.0,4.077272e-09,7.532681e-02,-0.901228,0.825902,4.565261e-02,-0.546199,0.500546,10.0,4.077272e-12
100003,10.0003,10000.0,1.425388e-09,1.128564e-01,-0.916921,0.804064,6.839781e-02,-0.555710,0.487312,10.0,1.425388e-12
100004,10.0004,10000.0,3.326995e-09,1.502256e-01,-0.931310,0.781085,9.104581e-02,-0.564430,0.473385,10.0,3.326995e-12
...,...,...,...,...,...,...,...,...,...,...,...
149996,14.9996,10000.0,2.822683e-09,-1.502256e-01,-0.781085,0.931310,-9.104581e-02,-0.473385,0.564430,10.0,2.822683e-12
149997,14.9997,10000.0,9.331541e-11,-1.128564e-01,-0.804064,0.916921,-6.839781e-02,-0.487312,0.555710,10.0,9.331541e-14
149998,14.9998,10000.0,-2.637924e-09,-7.532681e-02,-0.825902,0.901228,-4.565261e-02,-0.500546,0.546199,10.0,-2.637924e-12
149999,14.9999,10000.0,-5.344666e-09,-3.769018e-02,-0.846565,0.884255,-2.284253e-02,-0.513070,0.535912,10.0,-5.344666e-12


In [35]:
# Create an output directory if needed
output_dir = './'
os.makedirs(output_dir, exist_ok=True)

# Save each trimmed DataFrame
pf_all.to_csv(os.path.join(output_dir, 'pf_all.csv'), index=False)
pwr_all.to_csv(os.path.join(output_dir, 'pwr_all.csv'), index=False)
simulink_all.to_csv(os.path.join(output_dir, 'simulink_all.csv'), index=False)
